In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error, mean_absolute_percentage_error

## 1. Definimos nuestro problema

Dados dos dataset (train y test) con información de ventas de inmuebles, desarrollar un modelo a
partir del dataset train que permita predecir el valor aproximado/objetivo de los inmuebles en
el dataset test.

## 2. Construcción de nuestro dataset

*Consideraciones*:<br>
- Ambos datasets provistos constan de 80 features, pero test no contiene nuestra target feature, por lo que todos los steps de entrenamiento, validación y testeo lo haremos con train.<br>
- Test sólo será usado al final del proceso para el step de inference.<br>
- Train tiene 1460 registros, test tiene 1459.

In [2]:
# Importamos el dataset train, explicitamos 'MSSubClass' como variable categórica.
df = pd.read_csv('./Housing Dreams/house_train_raw.csv', dtype={'MSSubClass':str})
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
# Guardamos nuestro target en una variable y lo dropeamos junto con 'Id' del dataset.
target = df['SalePrice']
df = df.drop(['Id', 'SalePrice'], axis=1)

### 2.a. Pre-processing.

En ésta etapa crearemos pipelines para automatizar la limpieza y pre-procesamiento de los datos.<br>
*Consideraciones*:<br>
- En esta primera implementación me enfocaré en usar modelos 'tree-based' como pueden ser RandomForest o GradientBoosting, ya que no requieren de transformaciones adicionales (Scaling, OneHotEncoding) para performar adecuadamente.

Comenzamos por diferenciar las variables categóricas de las númericas.

In [4]:
cat_selector = make_column_selector(dtype_include=object)
num_selector = make_column_selector(dtype_include=np.number)
cat_selector(df)

['MSSubClass',
 'MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [5]:
num_selector(df)

['LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

El siguiente paso sería definir e implementar los pipelines de pre-procesamiento, cómo ya dijimos anteriormente, en ésta ocasión nos enfocaremos en modelos 'tree-based'. Por lo que nuestro pre-procesamiento se basará en imputar nulos y realizar ordinal encoding a las features categóricas.

In [6]:
cat_processor = OrdinalEncoder(
    handle_unknown='use_encoded_value', unknown_value=-1, encoded_missing_value=-1)

num_processor = SimpleImputer(strategy='median', add_indicator=False)

preprocessor = make_column_transformer(
    (num_processor, num_selector), (cat_processor, cat_selector)
)
preprocessor

ColumnTransformer(transformers=[('simpleimputer',
                                 SimpleImputer(strategy='median'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x0000018B12A479A0>),
                                ('ordinalencoder',
                                 OrdinalEncoder(encoded_missing_value=-1,
                                                handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x0000018B1330C400>)])

### 2.b. Feature Engineering.

En ésta etapa crearemos un pipeline que aplique *SelectKBest* para elegir las 20 features más performantes para nuestros modelos 'tree-based'.

In [7]:
fe_pipeline = make_pipeline(
    preprocessor,
    SelectKBest(score_func=f_regression, k=20)
)
fe_pipeline.fit(df, target)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(strategy='median'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000018B12A479A0>),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(encoded_missing_value=-1,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000018B1330C400>)])),
                ('selectkbest',
                 SelectKBest(k=20,
                             score_func=<function f_regression at 0x0000018B13063250>))])

Guardamos los nombres de las features elegidas para aplicarlo más tarde en el dataset 'Test' y aplicamos el filtro al df actual.

In [8]:
features_used = fe_pipeline.get_feature_names_out()
features_used = [feature.split('__')[1] for feature in features_used]
df1 = df[features_used]

## 3. Entrenamiento de los modelos.

Siguiendo la misma línea que en los pasos anteriores, se implementarán pipelines para entrenar 3 modelos 'tree-based':<br>
- RandomForestRegressor.<br>
- GradientBoostingRegressor.<br>
- AdaBoostRegressor.<br>
- BaggingRegressor.<br>
- XGBRegressor.<br>

### 3.a. Implementación de pipelines.

In [9]:
rf_pipeline = make_pipeline(preprocessor, RandomForestRegressor(random_state=420))
rf_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(strategy='median'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000018B12A479A0>),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(encoded_missing_value=-1,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000018B1330C400>)])),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=420))])

In [10]:
gb_pipeline = make_pipeline(preprocessor, GradientBoostingRegressor(random_state=420))
gb_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(strategy='median'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000018B12A479A0>),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(encoded_missing_value=-1,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000018B1330C400>)])),
                ('gradientboostingregressor',
                 GradientBoostingRegressor(random_state=420))])

In [11]:
ab_pipeline = make_pipeline(preprocessor, AdaBoostRegressor(random_state=420))
ab_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(strategy='median'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000018B12A479A0>),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(encoded_missing_value=-1,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000018B1330C400>)])),
                ('adaboostregressor', AdaBoostRegressor(random_state=420))])

In [12]:
bag_pipeline = make_pipeline(preprocessor, BaggingRegressor(random_state=420))
bag_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(strategy='median'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000018B12A479A0>),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(encoded_missing_value=-1,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000018B1330C400>)])),
                ('baggingregressor', BaggingRegressor(random_state=420))])

In [13]:
xgb_pipeline = make_pipeline(preprocessor, XGBRegressor(random_state=420))
xgb_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(strategy='median'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000018B12A479A0>),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(encoded_missing_value=-1,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transform...
                              gamma=None, gpu_id=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=None,
                              max_leaves=None, min_child_weight=None,
                              missing=nan, monotone_constraints=None,
                              n_estimators=100, n_jobs=None,
                              num_parallel_tree=None, predictor=None,
                              random_state=420, reg_alpha=None, reg_lambda=None, ...))])

### 3.b. Model training usando los pipelines.

*Consideraciones*:<br>
- En esta ocasión sólo divideremos nuestro df en 2: Train y Test, la implementación de Validation se hará en próximas versiones.

In [14]:
# Primero dividimos nuestro df en 2:
# Train: 80% - Test: 20%

X_train, X_test, y_train, y_test = train_test_split(df1, target, test_size=0.2, random_state=420)

In [15]:
# Entrenamos los modelos.
rf_pipeline.fit(X_train, y_train)

gb_pipeline.fit(X_train, y_train)

ab_pipeline.fit(X_train, y_train)

bag_pipeline.fit(X_train, y_train)

xgb_pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(strategy='median'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000018B12A479A0>),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(encoded_missing_value=-1,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transform...
                              gamma=0, gpu_id=-1, grow_policy='depthwise',
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.300000012, max_bin=256,
                              max_cat_to_onehot=4, max_delta_step=0,
                              max_depth=6, max_leaves=0, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=100, n_jobs=0, num_parallel_tree=1,
                              predictor='auto', random_state=420, reg_alpha=0,
                              reg_lambda=1, ...))])

## 4. Evaluación de los modelos.

In [16]:
# Realizamos todas las predicciones usando X_test.
y_preds_rf = rf_pipeline.predict(X_test)

y_preds_gb = gb_pipeline.predict(X_test)

y_preds_ab = ab_pipeline.predict(X_test)

y_preds_bag = bag_pipeline.predict(X_test)

y_preds_xgb = xgb_pipeline.predict(X_test)

In [17]:
# Calculamos la métrica requerida para cada uno de nuestros modelos
l_func_rf = mean_squared_log_error(y_test, y_preds_rf, squared=False)
l_func_gb = mean_squared_log_error(y_test, y_preds_gb, squared=False)
l_func_ab = mean_squared_log_error(y_test, y_preds_ab, squared=False)
l_func_bag = mean_squared_log_error(y_test, y_preds_bag, squared=False)
l_func_xgb = mean_squared_log_error(y_test, y_preds_xgb, squared=False)
print(f'Random Forest Reg: {l_func_rf}')
print(f'Gradient Boosting Reg: {l_func_gb}')  # Winner
print(f'Ada Boosting Reg: {l_func_ab}')
print(f'Bagging Reg: {l_func_bag}')
print(f'XGBReg: {l_func_xgb}')

Random Forest Reg: 0.15867256139270283
Gradient Boosting Reg: 0.1541758378950554
Ada Boosting Reg: 0.22134815046775233
Bagging Reg: 0.16215722210089042
XGBReg: 0.17152834439287726


In [18]:
# Calculamos el MAPE del modelo que mejor performó (GradientBoostingReg)
l_func_gb = mean_absolute_percentage_error(y_test, y_preds_gb)*100
l_func_gb  # 89.26% de accuracy

10.834294301503961

## 5. Model Inference.

En ésta etapa haremos las predicciones de nuestro segundo dataset y guardaremos las mismas en un formato CSV.

In [19]:
# Lectura del Dataset.
df_inf = pd.read_csv('./Housing Dreams/houses_test_raw.csv')

In [20]:
# Filtramos por las features elegidas usando SelectKBest.
X_inf = df_inf[features_used]

In [21]:
# Realizamos nuestras predicciones!.
y_inf = gb_pipeline.predict(X_inf)

In [22]:
# Acá transformamos nuestras predicciones de np.array a pd.DataFrame para poder usar el método .to_csv() para guardar nuestras predicciones en el formato requerido.
y_inf_df = pd.DataFrame(y_inf, columns=['pred'])
y_inf_df.to_csv('pred_test.csv', index=False)